In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../'))

from TumorDetection.Utils.WorkingDir import WorkingDir
WorkingDir.set_wd()
from time import perf_counter
import torch
import math
import numpy as np
import plotly.graph_objects as go

from TumorDetection.Utils.DictClasses import DataPathDir
from TumorDetection.Utils.Viewer import Viewer
from TumorDetection.Data.Loader import DataPathLoader, ImageLoader
from TumorDetection.Data.PatchDataset import PatchDataset
from TumorDetection.Models.EfficientNet.EfficientNet import EFSNetModel

In [7]:
dp = DataPathLoader(DataPathDir.get('dir_path'),
                    substring_filter='normal')
il = ImageLoader()(paths_classes=dp())

## Patched Batches Evaluation

In [3]:
_, X_val, _, y_val = PatchDataset()(images=[t[2] for t in il],
                                    masks=[t[-1] for t in il])
print('Validation data')
print(X_val.shape, y_val.shape)

Validation data
(1625, 256, 256, 1) (1625, 256, 256)


In [4]:
# perc_classes = {k: v/y_tr.size for k, v in np.asarray(np.unique(y_tr, return_counts=True)).T}
# perc_classes = [(idx, np.sum(y_val[...,idx])/y_val[...,idx].size) for idx in range(y_val.shape[-1])]
# print(perc_classes)
# print({k: 1/math.log(1.02+v) for k,v in perc_classes.items()})

In [5]:
perc_classes = {k: v/y_val.size for k, v in np.asarray(np.unique(y_val, return_counts=True)).T}
# perc_classes = [(idx, np.sum(y_val[...,idx])/y_val[...,idx].size) for idx in range(y_val.shape[-1])]
print(perc_classes)
print({k: 1/math.log(1.02+v) for k,v in perc_classes.items()})

{0: 0.836554208608774, 1: 0.06942348069411057, 2: 0.09402231069711539}
{0: 1.616234249227115, 1: 11.675608691446968, 2: 9.261217075377944}


In [6]:
print({k: (1/math.log(1.02+v))/(1/math.log(1.02+perc_classes[0])) for k,v in perc_classes.items()})

{0: 1.0, 1: 7.22395822080262, 2: 5.730120543978491}


In [7]:
efsnet = EFSNetModel(verbose=0)
efsnet.load_model()
y_pred_raw = efsnet.model.predict(X_val)
y_pred = np.argmax(y_pred_raw, axis = -1)

C:\Users\User\Git\TumorDetection\venv\lib\site-packages\keras\src\saving\saving_lib.py:388: UserWarning: Skipping variable loading for optimizer 'adam', because it has 1 variables whereas the saved optimizer has 529 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


51/51 ━━━━━━━━━━━━━━━━━━━━ 10s 168ms/step


In [8]:
perc_pred_classes = {k: v/y_pred.size for k, v in np.asarray(np.unique(y_pred, return_counts=True)).T}
# perc_pred_classes = [(idx, np.sum(y_pred_raw[...,idx])/y_pred_raw[...,idx].size) for idx in range(y_pred_raw.shape[-1])]
perc_pred_classes

{0: 0.7631473764272837, 1: 0.06336005108173078, 2: 0.17349257249098557}

In [10]:
idx = 200
print(y_val[idx].max())
print({k: v/y_pred[idx].size for k, v in np.asarray(np.unique(y_pred[idx], return_counts=True)).T})
y_pred_i = y_pred[idx].copy()
# y_pred_i[y_pred_i == 1] = 0
y_pred_i = y_pred_i*(255/y_pred.max())
print({k: v/y_pred_i.size for k, v in np.asarray(np.unique(y_pred_i, return_counts=True)).T})
x = (X_val[idx].squeeze()*255).astype(np.uint8)
Viewer.show_masked_image(x, y_val[idx], win_title='Real')
Viewer.show_masked_image(x, y_pred_i.astype(np.uint8), win_title='Prediction')
Viewer.show_masked_image(x, (y_val[idx]!=y_pred[idx]).astype(np.uint8), win_title='Difference')

1
{0: 0.8894195556640625, 1: 0.071136474609375, 2: 0.0394439697265625}
{0.0: 0.8894195556640625, 127.5: 0.071136474609375, 255.0: 0.0394439697265625}


In [28]:
from keras.metrics import MeanIoU
miou = MeanIoU(num_classes = efsnet.num_classes, ignore_class=0)
# print(miou())

In [30]:
ms = []
for i in range(len(y_val)):
    ms.append(miou(np.expand_dims(y_val[idx], axis = -1), np.expand_dims(y_pred[idx], axis = -1)).cpu().numpy())
print(np.mean(ms))

0.32768172


## Image Composition Evaluation

In [8]:
X_tr, X_val, y_tr, y_val = PatchDataset()(images=[t[2] for t in il],
                                          masks=[t[-1] for t in il],
                                          mode='images')
print('Train data')
print(X_tr.shape, y_tr.shape)
print('Validation data')
print(X_val.shape, y_val.shape)

Train data
(582, 512, 512) (582, 512, 512)
Validation data
(65, 512, 512) (65, 512, 512)


In [42]:
efsnet = EFSNetModel(verbose=0)
efsnet.load_model()

C:\Users\User\Git\TumorDetection\venv\lib\site-packages\keras\src\saving\saving_lib.py:388: UserWarning: Skipping variable loading for optimizer 'adam', because it has 1 variables whereas the saved optimizer has 529 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [43]:
from patchify import patchify, unpatchify
image = X_val[0]
batch = patchify(image, efsnet.model.input.shape[1:-1], 32)
batch_y = np.argmax(efsnet.model.predict(batch.reshape(-1, *efsnet.model.input.shape[1:-1], 1)),
                    axis = -1).reshape(batch.shape)
y_pred = unpatchify(batch_y, image.shape)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 328ms/step


In [44]:
x = (image.squeeze()*255).astype(np.uint8)
Viewer.show_masked_image(x, y_val[0], win_title='Real')
Viewer.show_masked_image(x, (y_pred*(255/y_pred.max())).astype(np.uint8), win_title='Prediction')
Viewer.show_masked_image(x, (y_val[0]!=y_pred[0]).astype(np.uint8), win_title='Difference')

In [45]:
yv = patchify(y_val[0], efsnet.model.input.shape[1:-1], 32)
for b, by, y in zip(batch.reshape(-1, *efsnet.model.input.shape[1:-1]), 
                 batch_y.reshape(-1, *efsnet.model.input.shape[1:-1]),
                 yv.reshape(-1, *efsnet.model.input.shape[1:-1])):
    x = (b.squeeze()*255).astype(np.uint8)
    Viewer.show_masked_image(x, y, win_title='Real')
    Viewer.show_masked_image(x, (by*(255/2)).astype(np.uint8), win_title='Prediction')
    Viewer.show_masked_image(x, (y!=by).astype(np.uint8), win_title='Difference')

In [41]:
efsnet.loaded

False

In [46]:
dp()

[('C:\\Users\\User\\Git\\TumorDetection\\resources\\Dataset_BUSI_with_GT\\benign\\benign (1).png',
  ['C:\\Users\\User\\Git\\TumorDetection\\resources\\Dataset_BUSI_with_GT\\benign\\benign (1)_mask.png'],
  ['benign']),
 ('C:\\Users\\User\\Git\\TumorDetection\\resources\\Dataset_BUSI_with_GT\\benign\\benign (10).png',
  ['C:\\Users\\User\\Git\\TumorDetection\\resources\\Dataset_BUSI_with_GT\\benign\\benign (10)_mask.png'],
  ['benign']),
 ('C:\\Users\\User\\Git\\TumorDetection\\resources\\Dataset_BUSI_with_GT\\benign\\benign (100).png',
  ['C:\\Users\\User\\Git\\TumorDetection\\resources\\Dataset_BUSI_with_GT\\benign\\benign (100)_mask.png',
   'C:\\Users\\User\\Git\\TumorDetection\\resources\\Dataset_BUSI_with_GT\\benign\\benign (100)_mask_1.png'],
  ['benign', 'benign']),
 ('C:\\Users\\User\\Git\\TumorDetection\\resources\\Dataset_BUSI_with_GT\\benign\\benign (101).png',
  ['C:\\Users\\User\\Git\\TumorDetection\\resources\\Dataset_BUSI_with_GT\\benign\\benign (101)_mask.png'],
  ['be